In [1]:
import sys
sys.path.insert(0,'/exp/rhuang/meta/icefall/egs/librispeech/ASR/conformer_ctc2_noseg')
sys.path.insert(0,'/exp/rhuang/meta/k2/k2/python')
sys.path.insert(0,'/exp/rhuang/meta/k2/temp.linux-x86_64-cpython-310/lib')
sys.path.insert(0,'/exp/rhuang/meta/icefall')

In [2]:
%cd /exp/rhuang/meta/icefall/egs/librispeech/ASR
%pwd  #look at the current work dir

/exp/rhuang/meta/icefall/egs/librispeech/ASR


'/exp/rhuang/meta/icefall/egs/librispeech/ASR'

In [28]:
import argparse
import logging
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import k2
import sentencepiece as spm
import torch
import torch.nn as nn
from asr_datamodule import LibriSpeechAsrDataModule
from conformer import Conformer

from icefall.bpe_graph_compiler import BpeCtcTrainingGraphCompiler
from icefall.checkpoint import (
    average_checkpoints,
    average_checkpoints_with_averaged_model,
    find_checkpoints,
    load_checkpoint,
)
from icefall.decode import (
    get_lattice,
    nbest_decoding,
    nbest_oracle,
    one_best_decoding,
    rescore_with_attention_decoder,
    rescore_with_n_best_list,
    rescore_with_rnn_lm,
    rescore_with_whole_lattice,
)
from icefall.env import get_env_info
from icefall.lexicon import Lexicon
from icefall.rnn_lm.model import RnnLmModel
from icefall.utils import (
    AttributeDict,
    get_texts,
    load_averaged_model,
    setup_logger,
    store_transcripts,
    str2bool,
    write_error_stats,
)
# from train import get_params as get_train_params
from decode import get_parser, get_params
from icefall.decode import get_lattice, one_best_decoding
from icefall.utils import get_alignments, get_texts, get_texts_with_timestamp

import psutil
try:
    from tqdm_loggable.auto import tqdm
except:
    from tqdm import tqdm
import torch.multiprocessing as mp

In [20]:
parser = get_parser()
LibriSpeechAsrDataModule.add_arguments(parser)
args = parser.parse_args(args=[])

args.exp_dir = "/exp/rhuang/meta/icefall/egs/librispeech/ASR/conformer_ctc2_noseg/exp/exp_seed"
args.epoch = 14
args.avg = 1
args.use_averaged_model = False

args.manifest_dir = "/exp/rhuang/meta/icefall/egs/librispeech/ASR/data/fbank/"
args.max_duration = 30
args.world_size = 1
args.method = 'ctc-decoding'

args.exp_dir = Path(args.exp_dir)
args.lang_dir = Path(args.lang_dir)
args.lm_dir = Path(args.lm_dir)
args.manifest_dir = Path(args.manifest_dir)

params = get_params()
params.update(vars(args))

device = torch.device("cpu")

sp = spm.SentencePieceProcessor()
sp.load(params.bpe_model)

# <blk> is defined in local/train_bpe_model.py
params.blank_id = sp.piece_to_id("<blk>")
params.vocab_size = sp.get_piece_size()

In [15]:
lexicon = Lexicon(params.lang_dir)
max_token_id = max(lexicon.tokens)
num_classes = max_token_id + 1  # +1 for the blank

if params.method == "ctc-decoding" or params.method == "ctc-greedy-search":
    HLG = None
    H = k2.ctc_topo(
        max_token=max_token_id,
        modified=False,
        device=device,
    )
    bpe_model = spm.SentencePieceProcessor()
    bpe_model.load(str(params.lang_dir / "bpe.model"))
else:
    H = None
    bpe_model = None
    HLG = k2.Fsa.from_dict(
        torch.load(f"{params.lang_dir}/HLG.pt", map_location=device)
    )
    assert HLG.requires_grad is False

    if not hasattr(HLG, "lm_scores"):
        HLG.lm_scores = HLG.scores.clone()

print(f"H: {H is not None}")
print(f"HLG: {HLG is not None}")

H: True
HLG: False


In [16]:
print(params)

{'subsampling_factor': 4, 'feature_dim': 80, 'nhead': 8, 'dim_feedforward': 2048, 'encoder_dim': 512, 'num_encoder_layers': 12, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'env_info': {'k2-version': '1.24.3', 'k2-build-type': 'Debug', 'k2-with-cuda': True, 'k2-git-sha1': '2b2ac14b326d61d79d04e53fbd69b1ff6d630411', 'k2-git-date': 'Wed Aug 23 17:58:26 2023', 'lhotse-version': '1.20.0.dev+git.c863fa72.clean', 'torch-version': '2.0.1', 'torch-cuda-available': False, 'torch-cuda-version': '11.7', 'python-version': '3.1', 'icefall-git-branch': 'context', 'icefall-git-sha1': 'b805020-dirty', 'icefall-git-date': 'Thu Feb 8 10:25:35 2024', 'icefall-path': '/exp/rhuang/meta/icefall', 'k2-path': '/exp/rhuang/meta/k2/k2/python/k2/__init__.py', 'lhotse-path': '/exp/rhuang/meta/lhotse/lhotse/__init__.py', 'hostname': 'test2', 'IP address': '10.1.250.202'}, 'blank_id': 0, 'vocab_size': 500, 'epoch': 14, 'iter': 0, 'avg': 1, 'met

In [7]:
# params = AttributeDict(
#     {'best_train_loss': float('inf'), 'best_valid_loss': float('inf'), 'best_train_epoch': -1, 'best_valid_epoch': -1, 'batch_idx_train': 0, 'log_interval': 50, 'reset_interval': 200, 'valid_interval': 3000, 'feature_dim': 80, 'subsampling_factor': 4, 'warm_step': 2000, 'env_info': {'k2-version': '1.24.4', 'k2-build-type': 'Debug', 'k2-with-cuda': True, 'k2-git-sha1': '415fe1f446fffe1d9e7219b5033966294c0b430c', 'k2-git-date': 'Wed Dec 20 21:38:57 2023', 'lhotse-version': '1.20.0.dev+git.c6788492.clean', 'torch-version': '2.1.2', 'torch-cuda-available': True, 'torch-cuda-version': '11.8', 'python-version': '3.1', 'icefall-git-branch': 'noseg', 'icefall-git-sha1': 'b0ecb7f-dirty', 'icefall-git-date': 'Thu Feb 1 03:39:11 2024', 'icefall-path': '/scratch4/skhudan1/rhuang25/icefall', 'k2-path': '/scratch4/skhudan1/rhuang25/k2/k2/python/k2/__init__.py', 'lhotse-path': '/scratch4/skhudan1/rhuang25/lhotse/lhotse/__init__.py', 'hostname': 'gpu02', 'IP address': '172.20.30.2'}, 'frame_shift_ms': 10, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'epoch': 1, 'iter': 0, 'avg': 1, 'use_averaged_model': False, 'exp_dir': Path('zipformer_no_seg/exp-test'), 'bpe_model': 'data/lang_bpe_500/bpe.model', 'lang_dir': Path('data/lang_bpe_500'), 'context_size': 2, 'decoding_method': '1best', 'num_paths': 100, 'nbest_scale': 1.0, 'hlg_scale': 0.6, 'lm_dir': Path('data/lm'), 'num_encoder_layers': '2,2,3,4,3,2', 'downsampling_factor': '1,2,4,8,4,2', 'feedforward_dim': '512,768,1024,1536,1024,768', 'num_heads': '4,4,4,8,4,4', 'encoder_dim': '192,256,384,512,384,256', 'query_head_dim': '32', 'value_head_dim': '12', 'pos_head_dim': '4', 'pos_dim': 48, 'encoder_unmasked_dim': '192,192,256,256,256,192', 'cnn_module_kernel': '31,31,15,15,15,31', 'decoder_dim': 512, 'joiner_dim': 512, 'causal': False, 'chunk_size': '16,32,64,-1', 'left_context_frames': '64,128,256,-1', 'use_transducer': False, 'use_ctc': True, 'full_libri': True, 'mini_libri': False, 'manifest_dir': Path('data/fbank'), 'max_duration': 500, 'bucketing_sampler': True, 'num_buckets': 30, 'concatenate_cuts': False, 'duration_factor': 1.0, 'gap': 1.0, 'on_the_fly_feats': False, 'shuffle': True, 'drop_last': True, 'return_cuts': True, 'num_workers': 2, 'enable_spec_aug': True, 'spec_aug_time_warp_factor': 80, 'enable_musan': True, 'input_strategy': 'PrecomputedFeatures', 'res_dir': Path('zipformer_no_seg/exp-test/1best'), 'suffix': 'epoch-1-avg-1', 'vocab_size': 500, 'blank_id': 0}
# )

In [17]:
logging.info("About to create model")
model = Conformer(
    num_features=params.feature_dim,
    nhead=params.nhead,
    d_model=params.encoder_dim,
    num_classes=params.vocab_size,
    subsampling_factor=params.subsampling_factor,
    num_encoder_layers=params.num_encoder_layers,
    num_decoder_layers=params.num_decoder_layers,
)

load_checkpoint(f"{params.exp_dir}/epoch-{params.epoch}.pt", model)
model.to(device)

num_param = sum([p.numel() for p in model.parameters()])
logging.info(f"Number of model parameters: {num_param}")

In [61]:
# _ = model.eval()
_ = model.train()

In [21]:
args.return_cuts = True
librispeech = LibriSpeechAsrDataModule(args)

train_cuts = librispeech.train_clean_100_cuts()
# if params.full_libri:
#     train_cuts += librispeech.train_clean_360_cuts()
#     train_cuts += librispeech.train_other_500_cuts()
# train_cuts = librispeech.train_other_500_cuts()

# valid_cuts = librispeech.dev_clean_cuts()
# valid_cuts += librispeech.dev_other_cuts()

# test_cuts = librispeech.test_clean_cuts()
# test_cuts += librispeech.test_other_cuts()
# test_cuts = test_cuts.to_eager()

In [22]:
c = train_cuts.sample(n_cuts=1)
# c = test_cuts['1320-122612-0011-2251']
c

MonoCut(id='8629-261139-0012-6287_sp0.9', start=0.0, duration=15.82775, channel=0, supervisions=[SupervisionSegment(id='8629-261139-0012_sp0.9', recording_id='8629-261139-0012_sp0.9', start=0.0, duration=15.82775, channel=0, text='WITH A FELL AND STEADY PURPOSE HE FOUND HIS RIGHT HAND STEALING TOWARD THE LEFT IN THE SIGNIFICANT ACTION SHE EXPECTED BETTER TO YIELD THAN FALL HEADLONG INTO THE PIT ONE WORD OF HERS WOULD OPEN', language='English', speaker='8629', gender=None, custom=None, alignment=None)], features=Features(type='kaldi-fbank', num_frames=1583, num_features=80, frame_shift=0.01, sampling_rate=16000, start=0.0, duration=15.82775, storage_type='lilcom_chunky', storage_path='data/fbank/librispeech_feats_train-clean-100/feats-11.lca', storage_key='277261067,48541,49009,48114,8292', recording_id='None', channels=0), recording=Recording(id='8629-261139-0012_sp0.9', sources=[AudioSource(type='file', channels=[0], source='/exp/rhuang/librispeech/download2/LibriSpeech/train-clean-10

In [23]:
c.play_audio()

In [24]:
def get_uid_key(my_id):
    # /data/skhudan1/corpora/librispeech/CHAPTERS.TXT
    speaker_id, chapter_id, utterance_id, _ = my_id.split("-")
    speaker_id, chapter_id, utterance_id = int(speaker_id), int(chapter_id), int(utterance_id)
    return speaker_id, chapter_id, utterance_id


def convert_long_text_to_fst(items, sp):
    libri_long_text_sp = dict()
    for k, text in tqdm(items, mininterval=2, desc=f"libri_long_text"):
        libri_long_text_sp[k] = make_factor_transducer1(sp.encode(text, out_type=int), return_str=True, blank_penalty=0)
        # libri_long_text_sp[k] = make_factor_transducer2(sp.encode(text, out_type=int), return_str=True, blank_penalty=-12)
    return libri_long_text_sp


def get_long_text(cuts, sp=None, make_fst=False):
    logging.info(f"Getting long text from cuts ... ")  # len(cuts) = {len(cuts)}
    cuts_by_recoding = defaultdict(list)
    for i, c in enumerate(cuts):  # tqdm(cuts, miniters=1000, total=None):
        # if "_sp" in c.id:
        #     continue
        cuts_by_recoding[tuple(get_uid_key(c.id)[:2])].append(c)

        # if i % 1e4 == 0:
        #     print(f"progress: {i}")

    libri_long_text = dict()
    for k, v in cuts_by_recoding.items():
        v.sort(key = lambda x: get_uid_key(x.id)[-1])
        text = " ".join([c.supervisions[0].text for c in v])
        libri_long_text[k] = text
    
    if sp is None:
        return libri_long_text

    logging.info(f"Converting long text to fst ... ")

    if not make_fst:
        libri_long_text_sp = dict()
        for k, text in libri_long_text.items():
            libri_long_text_sp[k] = sp.encode(text, out_type=int)
    else:
        results = [0]
        libri_long_text_sp = convert_long_text_to_fst(
            libri_long_text.items(),
            sp,
        )
        for k, v in tqdm(libri_long_text_sp.items()):
            libri_long_text_sp[k] = k2.Fsa.from_str(v, acceptor=False)
    
    ram_info = psutil.virtual_memory()
    ram_used_mb = ram_info.used / (1024 ** 2)  # Convert bytes to megabytes
    ram_total_mb = ram_info.total / (1024 ** 2)  # Convert bytes to megabytes
    ram_usage = ram_info.percent
    logging.info(f"Current RAM Usage: {ram_used_mb:.2f} MB out of {ram_total_mb:.2f} MB ({ram_usage}%)")

    return libri_long_text_sp


def make_factor_transducer1(word_id_list, return_str=False, blank_penalty=0):
    # This is the original, simplest factor transducer for a "linear" fst

    fst_graph = k2.ctc_graph([word_id_list], modified=False, device='cpu')[0]

    c_str = k2.to_str_simple(fst_graph)
    arcs = c_str.strip().split("\n")
    arcs = [x.strip() for x in arcs if len(x.strip()) > 0]
    final_state = int(arcs[-1])
    
    arcs = arcs[:-1]
    arcs = [tuple(map(int, a.split())) for a in arcs]
    # ss, ee, l1, l2, w = arc

    non_eps_nodes = set((arc[1], arc[3]) for arc in arcs if arc[3] > -1)   # if this node has a non-eps in-coming arc
    arcs += [(0, n, l, l, 0) for n, l in non_eps_nodes if n > 1]

    arcs += [(n, final_state, -1, -1, 0) for n in range(1, final_state - 2)]

    new_arcs = arcs
    new_arcs.append([final_state])

    new_arcs = sorted(new_arcs, key=lambda arc: arc[0])
    new_arcs = [[str(i) for i in arc] for arc in new_arcs]
    new_arcs = [" ".join(arc) for arc in new_arcs]
    new_arcs = "\n".join(new_arcs)

    if return_str:
        return new_arcs
    else:
        fst = k2.Fsa.from_str(new_arcs, acceptor=False)
        return fst


def make_factor_transducer2(word_id_list, return_str=False, blank_penalty=-1):
    # This is the factor transducer where blank symbols at the beginning and ending of the graph is penalized
    # Last resort: use a cheap alignment model to get a subgraph of the big graph first

    # blank_penalty should be negative

    fst_graph = k2.ctc_graph([word_id_list], modified=False, device='cpu')[0]

    c_str = k2.to_str_simple(fst_graph)
    arcs = c_str.strip().split("\n")
    arcs = [x.strip() for x in arcs if len(x.strip()) > 0]
    final_state = int(arcs[-1])
    
    arcs = arcs[:-1]
    arcs = [tuple(map(int, a.split())) for a in arcs]
    # ss, ee, l1, l2, w = arc

    arc0 = arcs[0]
    arcs_last = [a for a in arcs[-5:] if a[2] > 0]

    arcs = [(0, 0, 0, 0, blank_penalty)] + arcs[1:-5] + arcs_last

    non_eps_nodes = set((arc[1], arc[3]) for arc in arcs if arc[3] > 0)   # if this node has a non-eps in-coming arc
    arcs += [(0, n, l, l, 0) for n, l in non_eps_nodes if n > 1]

    # arcs += [(n, final_state, -1, -1, 0) for n in range(1, final_state - 2)]
    arcs += [(n, final_state - 1, 0, 0, blank_penalty) for n, l in non_eps_nodes]
    arcs += [(final_state - 1, final_state - 1, 0, 0, blank_penalty)]
    arcs += [(final_state - 1, final_state, -1, -1, 0)]

    new_arcs = arcs
    new_arcs.append([final_state])

    new_arcs = sorted(new_arcs, key=lambda arc: arc[0])
    new_arcs = [[str(i) for i in arc] for arc in new_arcs]
    new_arcs = [" ".join(arc) for arc in new_arcs]
    new_arcs = "\n".join(new_arcs)

    if return_str:
        return new_arcs
    else:
        fst = k2.Fsa.from_str(new_arcs, acceptor=False)
        return fst


def make_factor_transducer3(word_id_list, word_start_symbols, return_str=False, blank_penalty=0):
    # This is a modification of make_factor_transducer1, where the factors are on "word-level"
    # That is, the words always come as a whole

    fst_graph = k2.ctc_graph([word_id_list], modified=False, device='cpu')[0]

    c_str = k2.to_str_simple(fst_graph)
    arcs = c_str.strip().split("\n")
    arcs = [x.strip() for x in arcs if len(x.strip()) > 0]
    final_state = int(arcs[-1])
    
    arcs = arcs[:-1]
    arcs = [tuple(map(int, a.split())) for a in arcs]
    # ss, ee, l1, l2, w = arc

    non_eps_nodes = set((arc[1], arc[3]) for arc in arcs if arc[3] > 0 and arc[3] in word_start_symbols)   # if this node has a non-eps, word-start in-coming arc
    arcs += [(0, n, l, l, 0) for n, l in non_eps_nodes if n > 1]

    non_eps_nodes2 = set((arc[0], arc[3]) for arc in arcs if arc[3] > 0 and arc[3] in word_start_symbols)   # if this node has a non-eps, word-start out-going arc
    non_eps_nodes2 = [(n, l) for n, l in non_eps_nodes2 if 0 < n < final_state - 2]
    arcs += [(n, final_state, -1, -1, 0) for n, l in non_eps_nodes2]

    new_arcs = arcs
    new_arcs.append([final_state])

    new_arcs = sorted(new_arcs, key=lambda arc: arc[0])
    new_arcs = [[str(i) for i in arc] for arc in new_arcs]
    new_arcs = [" ".join(arc) for arc in new_arcs]
    new_arcs = "\n".join(new_arcs)

    if return_str:
        return new_arcs
    else:
        fst = k2.Fsa.from_str(new_arcs, acceptor=False)
        return fst


class MyCounter: 
    def __init__(self): self.counter1 = 0; self.counter2 = 1; self.counter2_ = 0
    # def __call__(self): self.counter += 1; return self.counter
    def f1(self): 
        self.counter1 += 1; return self.counter1
    def f2(self): 
        self.counter2_ += 1; 
        if self.counter2_ % 2 == 1: self.counter2 += 1; 
        return self.counter2


def make_factor_transducer4(word_id_list, word_start_symbols, return_str=False, blank_penalty=0):
    fst_graph = k2.ctc_graph([word_id_list], modified=False, device='cpu')[0]

    c_str = k2.to_str_simple(fst_graph)
    arcs = c_str.strip().split("\n")
    arcs = [x.strip() for x in arcs if len(x.strip()) > 0]
    final_state = int(arcs[-1])
    
    arcs = arcs[:-1]
    arcs = [tuple(map(int, a.split())) for a in arcs]
    # ss, ee, l1, l2, w = arc

    counter = MyCounter()

    non_eps_nodes1 = set((arc[1], arc[3]) for arc in arcs if arc[3] > 0 and arc[3] in word_start_symbols)   # if this node has a non-eps, word-start in-coming arc
    non_eps_nodes1 = sorted(non_eps_nodes1, key=lambda x: x[0])
    non_eps_nodes2 = list((arc[0], arc[1]) for arc in arcs if arc[3] < 0 or (arc[3] > 0 and arc[3] in word_start_symbols and arc[0] > 0))   # if this node has a non-eps, word-start out-going arc
    self_loops = {ss: l1 for ss, ee, l1, l2, w in arcs if ss == ee}

    arcs = [arcs[0]] + arcs[2:-5] + [a for a in arcs[-5:] if a[2] >= 0]
    arcs = [[ss, ee, l1, 0, w] for ss, ee, l1, l2, w in arcs]
    arcs += [(0, n, l, counter.f1(), 0) for n, l in non_eps_nodes1]
    arcs += [(n, final_state, self_loops[n], counter.f2(), 0) for n, l in non_eps_nodes2]
    arcs += [(final_state, final_state + 1, -1, -1, 0)]
    
    # non_eps_nodes2 = [(n, l) for n, l in non_eps_nodes2 if 0 < n < final_state - 2]
    
    new_arcs = arcs
    new_arcs.append([final_state + 1])

    new_arcs = sorted(new_arcs, key=lambda arc: arc[0])
    new_arcs = [[str(i) for i in arc] for arc in new_arcs]
    new_arcs = [" ".join(arc) for arc in new_arcs]
    new_arcs = "\n".join(new_arcs)

    # print(new_arcs)

    if return_str:
        return new_arcs
    else:
        fst = k2.Fsa.from_str(new_arcs, acceptor=False)
        return fst

In [29]:
libri_long_text = get_long_text([c], sp=sp, make_fst=True)
print(libri_long_text)
fst = list(libri_long_text.items())[0][1]
print(fst.shape[0], fst.num_arcs)

100%|██████████| 1/1 [00:00<00:00, 360.46it/s]

{(8629, 261139): <k2.fsa.Fsa object at 0x2aab934958a0>}
134 658


In [47]:
feature = torch.Tensor(c.load_features()).unsqueeze(0)
batch = {
    "supervisions": {
        "text": [c.supervisions[0].text],
        "cut": [c],
        "sequence_idx": torch.arange(1),
        "start_frame": torch.zeros(1),
        "num_frames": torch.Tensor([feature.size(1)]).int(),
    },
    "inputs": feature,
}

In [48]:
feature = batch["inputs"]
assert feature.ndim == 3

supervisions = batch["supervisions"]
feature_lens = torch.Tensor([feature.size(1)]).int()
print(feature_lens, feature.shape)

tensor([1583], dtype=torch.int32) torch.Size([1, 1583, 80])


In [49]:
# texts = batch["supervisions"]["text"]
# y = sp.encode(texts, out_type=int)
# y = k2.RaggedTensor(y)

cuts = batch['supervisions']['cut']
y_list = [libri_long_text[tuple(get_uid_key(c.id)[:2])] for c in cuts]
# y_list = [libri_long_text[(6415, 100596)] for c in cuts]  # A mismatched text

In [50]:
targets = y_list
x = feature
x_lens = feature_lens

In [62]:
ctc_output, memory, memory_key_padding_mask = model(feature, supervisions)  # ctc_output is (N, T, C)
encoder_out_lens = torch.div(
    supervisions["num_frames"],
    params.subsampling_factor,
    rounding_mode="floor",
) - 1  # to fix some issues -- just discard some frames

In [52]:
ctc_output.argmax(dim=-1)

tensor([[  0,   0,   0,   0,   0,  55,   0,   0,   0,   5,   0,   0,   0,   0,
         157,   0,   0,  58,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           7,   0,   0,   0,   0,   0, 109,  11,  11, 136,  16,  16,   0,   0,
          59,  79,  79,   0,  21,  21,  24,  77,  77,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,  22,   0,   0,   0, 396,   0,
           0,   0,   0,   0,   0,  46,   0,   0,   0,   0, 409,   0,   0,   0,
           0,   0,   0,   0, 273, 273,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0, 109,  11,  11,  48,   0,  14,  14,   0,   0,   0,   0,
          10,  10, 311, 311,   0,   0,   0,   0,   4,   4,   0,   0, 423,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,  15,   0,   0,   0,   0,   0,   0,   4,   0,   0,   0,
          35,   0,   0, 377, 377,  27,  27,  38,  38

In [55]:
input_lengths = encoder_out_lens
batch_size = input_lengths.size(0)
# TODO
supervision_segments = torch.stack(
    (
        torch.arange(batch_size),
        torch.zeros(batch_size),
        input_lengths.cpu(),
    ),
    1,
).to(torch.int32)
indices = torch.argsort(supervision_segments[:, 2], descending=True)
supervision_segments = supervision_segments[indices]

In [57]:
y_list = targets
_y_list = [y_list[i] for i in indices.tolist()]
decoding_graph = k2.create_fsa_vec(_y_list)
decoding_graph = k2.arc_sort(decoding_graph)
decoding_graph = decoding_graph.to(device)
decoding_graph.shape[0], decoding_graph.num_arcs

(1, 658)

In [58]:
log_probs = ctc_output
lattice = get_lattice(
    nnet_output=log_probs,
    decoding_graph=decoding_graph,
    supervision_segments=supervision_segments,
    search_beam=15,
    output_beam=6,
    min_active_states=30,
    max_active_states=10000,
    subsampling_factor=params.subsampling_factor,
)
lattice.shape[0], lattice.num_arcs

(1, 2628)

In [59]:
best_path = one_best_decoding(
    lattice=lattice,
    use_double_scores=True,
)

In [60]:
hyps = get_texts(best_path)
print(f"[ref]: {c.supervisions[0].text}")
print(f"[hyp]: {sp.decode(hyps)[0]}")

[ref]: WITH A FELL AND STEADY PURPOSE HE FOUND HIS RIGHT HAND STEALING TOWARD THE LEFT IN THE SIGNIFICANT ACTION SHE EXPECTED BETTER TO YIELD THAN FALL HEADLONG INTO THE PIT ONE WORD OF HERS WOULD OPEN
[hyp]: WITH A FELL AND STEADY PURPOSE HE FOUND HIS RIGHT HAND STEALING TOWARD THE LEFT IN THE SIGNIFICANT ACTION SHE EXPECTED BETTER TO YIELD THAN FALL HEADLONG INTO THE PIT ONE WORD OF HERS WOULD OPEN


In [26]:
# https://github.com/k2-fsa/icefall/tree/master/egs/librispeech/ASR/conformer_ctc
# https://github.com/k2-fsa/icefall/blob/master/icefall/ali.py
# https://github.com/k2-fsa/icefall/pull/1482
# https://github.com/k2-fsa/icefall/pull/876

labels_ali = get_alignments(best_path, kind="labels")
aux_labels_ali = get_alignments(best_path, kind="aux_labels")

In [28]:
decoding_results = get_texts_with_timestamp(best_path)

In [31]:
decoding_results.timestamps